<a href="https://colab.research.google.com/github/kl3778/APAN5400_Group/blob/Data-Analytics/DA_Pipeline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving APAN5400_Group-data_storage.zip to APAN5400_Group-data_storage.zip


In [3]:
import zipfile, os

zip_path = "/content/APAN5400_Group-data_storage.zip"  # update if different

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/group_storage")

# Check the contents
os.listdir("/content/group_storage")

['APAN5400_Group-data_storage']

In [4]:
os.listdir("/content/group_storage/APAN5400_Group-data_storage")

['.ipynb_checkpoints',
 'Pipeline2.ipynb',
 'kworb',
 'TL.ipynb',
 'Pipeline1.ipynb',
 'README.md']

In [5]:
os.listdir("/content/group_storage/APAN5400_Group-data_storage/kworb")

['spotify_artist_top10_albums_20251201.json',
 'spotify_artist_top10_albums_20251120.json',
 'spotify_artist_top10_albums_20251130.json',
 '.ipynb_checkpoints',
 'spotify_artist_top10_albums_20251203.json',
 'spotify_artist_top10_albums_20251202.json',
 'spotify_us_daily_artists_20251120.json',
 'spotify_artist_top10_ablums_20251204.csv',
 'spotify_monthly_listeners_20251120.json']

In [6]:
base_path = "/content/group_storage/APAN5400_Group-data_storage/kworb"

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("pipeline1") \
    .getOrCreate()

spark

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pipeline1").getOrCreate()

In [11]:
import os

base_path = "/content/group_storage/APAN5400_Group-data_storage/kworb"
os.listdir(base_path)

['spotify_artist_top10_albums_20251201.json',
 'spotify_artist_top10_albums_20251120.json',
 'spotify_artist_top10_albums_20251130.json',
 '.ipynb_checkpoints',
 'spotify_artist_top10_albums_20251203.json',
 'spotify_artist_top10_albums_20251202.json',
 'spotify_us_daily_artists_20251120.json',
 'spotify_artist_top10_ablums_20251204.csv',
 'spotify_monthly_listeners_20251120.json']

In [12]:
from pyspark.sql.functions import col

albums_7d = (
    spark.read
         .option("multiLine", True)
         .json(f"{base_path}/spotify_artist_top10_albums_*.json")
)

albums_7d.printSchema()
albums_7d.show(5, truncate=False)

root
 |-- album_title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_url: string (nullable = true)
 |-- daily: string (nullable = true)
 |-- streams: string (nullable = true)

+-------------------------+----------------------+-----------+-------------------------------------------------------------------+----------+--------------+
|album_title              |artist_id             |artist_name|artist_url                                                         |daily     |streams       |
+-------------------------+----------------------+-----------+-------------------------------------------------------------------+----------+--------------+
|^ The Highlights (Deluxe)|1Xyo4u8uXC1ZmMpatF05PJ|The Weeknd |https://kworb.net/spotify/artist/1Xyo4u8uXC1ZmMpatF05PJ_albums.html|21,763,803|50,459,335,647|
|^ The Highlights         |1Xyo4u8uXC1ZmMpatF05PJ|The Weeknd |https://kworb.net/spotify/artist/1Xyo4u8uXC1ZmMpatF05PJ_alb

In [16]:
import glob

base_path = "/content/group_storage/APAN5400_Group-data_storage/kworb"

files = glob.glob(f"{base_path}/spotify_artist_top10_albums_*.json")
print(files)  # see the filenames

first_file = files[0]
print("First file:", first_file)

# Look at the first 40 lines
with open(first_file, "r", encoding="utf-8") as f:
    for i in range(40):
        line = f.readline()
        if not line:
            break
        print(line.rstrip())

['/content/group_storage/APAN5400_Group-data_storage/kworb/spotify_artist_top10_albums_20251201.json', '/content/group_storage/APAN5400_Group-data_storage/kworb/spotify_artist_top10_albums_20251120.json', '/content/group_storage/APAN5400_Group-data_storage/kworb/spotify_artist_top10_albums_20251130.json', '/content/group_storage/APAN5400_Group-data_storage/kworb/spotify_artist_top10_albums_20251203.json', '/content/group_storage/APAN5400_Group-data_storage/kworb/spotify_artist_top10_albums_20251202.json']
First file: /content/group_storage/APAN5400_Group-data_storage/kworb/spotify_artist_top10_albums_20251201.json
[
  {
    "artist_name":"The Weeknd",
    "artist_id":"1Xyo4u8uXC1ZmMpatF05PJ",
    "album_title":"^ The Highlights (Deluxe)",
    "streams":"50,397,039,813",
    "daily":"20,767,272",
    "artist_url":"https:\/\/kworb.net\/spotify\/artist\/1Xyo4u8uXC1ZmMpatF05PJ_albums.html"
  },
  {
    "artist_name":"The Weeknd",
    "artist_id":"1Xyo4u8uXC1ZmMpatF05PJ",
    "album_title":

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, regexp_extract, to_date, col

spark = SparkSession.builder.appName("pipeline1").getOrCreate()

base_path = "/content/group_storage/APAN5400_Group-data_storage/kworb"

albums_7d = (
    spark.read
         .option("multiLine", True)   # required for JSON arrays = YOUR CASE!!
         .json(f"{base_path}/spotify_artist_top10_albums_*.json")
         .withColumn("source_file", input_file_name())
         .withColumn(
             "date",
             regexp_extract(col("source_file"),
                 r"spotify_artist_top10_albums_(\d{8})",  # matches 20251201
             1)
         )
         .withColumn("date", to_date("date", "yyyyMMdd"))
)

albums_7d.printSchema()
albums_7d.show(5, truncate=False)

root
 |-- album_title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_url: string (nullable = true)
 |-- daily: string (nullable = true)
 |-- streams: string (nullable = true)
 |-- source_file: string (nullable = false)
 |-- date: date (nullable = true)

+-------------------------+----------------------+-----------+-------------------------------------------------------------------+----------+--------------+---------------------------------------------------------------------------------------------------------+----------+
|album_title              |artist_id             |artist_name|artist_url                                                         |daily     |streams       |source_file                                                                                              |date      |
+-------------------------+----------------------+-----------+-------------------------------------------------------------

In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pipeline1_analytics").getOrCreate()

base_path = "/content/group_storage/APAN5400_Group-data_storage/kworb"

In [19]:
from pyspark.sql.functions import input_file_name, regexp_extract, to_date, col

albums_raw = (
    spark.read
         .option("multiLine", True)  # IMPORTANT for your JSON arrays
         .json(f"{base_path}/spotify_artist_top10_albums_*.json")
         .withColumn("source_file", input_file_name())
         .withColumn(
             "date_str",
             regexp_extract(
                 col("source_file"),
                 r"spotify_artist_top10_albums_(\d{8})",  # e.g. 20251201
                 1
             )
         )
         .withColumn("date", to_date("date_str", "yyyyMMdd"))
)

albums_raw.printSchema()
albums_raw.show(5, truncate=False)

root
 |-- album_title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_url: string (nullable = true)
 |-- daily: string (nullable = true)
 |-- streams: string (nullable = true)
 |-- source_file: string (nullable = false)
 |-- date_str: string (nullable = false)
 |-- date: date (nullable = true)

+-------------------------+----------------------+-----------+-------------------------------------------------------------------+----------+--------------+---------------------------------------------------------------------------------------------------------+--------+----------+
|album_title              |artist_id             |artist_name|artist_url                                                         |daily     |streams       |source_file                                                                                              |date_str|date      |
+-------------------------+----------------------+-----------+--

In [21]:
from pyspark.sql.functions import regexp_replace, col

albums_clean = (
    albums_raw
    .select(
        col("artist_name"),
        col("artist_id"),
        col("album_title").alias("album_name"),
        regexp_replace("streams", ",", "").cast("long").alias("streams_total"),
        regexp_replace("daily", ",", "").cast("long").alias("streams_daily"),
        col("date")
    )
    .dropna(subset=["artist_name", "artist_id", "album_name", "date"])
)

albums_clean.show(5, truncate=False)

+-----------+----------------------+-------------------------+-------------+-------------+----------+
|artist_name|artist_id             |album_name               |streams_total|streams_daily|date      |
+-----------+----------------------+-------------------------+-------------+-------------+----------+
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|^ The Highlights (Deluxe)|50459335647  |21763803     |2025-12-03|
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|^ The Highlights         |35380309196  |13535899     |2025-12-03|
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|^ The Weeknd In Japan    |18734326556  |6787192      |2025-12-03|
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|Starboy (Deluxe)         |18087019111  |6598588      |2025-12-03|
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|Starboy                  |16663942730  |6057126      |2025-12-03|
+-----------+----------------------+-------------------------+-------------+-------------+----------+
only showing top 5 rows



In [22]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

w = Window.partitionBy("date").orderBy(col("streams_daily").desc())

albums_ranked = (
    albums_clean
    .withColumn("rank", dense_rank().over(w))
    .filter(col("rank") <= 10)
)

albums_ranked.orderBy("date", "rank").show(20, truncate=False)

+-----------------------+----------------------+------------------------------------------------------------------------+-------------+-------------+----------+----+
|artist_name            |artist_id             |album_name                                                              |streams_total|streams_daily|date      |rank|
+-----------------------+----------------------+------------------------------------------------------------------------+-------------+-------------+----------+----+
|Michael Jackson        |3fMbdgg4jU18AjLCKBhRSm|^ The Essential Michael Jackson                                         |9782215374   |184839387    |2025-11-20|1   |
|Michael Jackson        |3fMbdgg4jU18AjLCKBhRSm|^ The Essential Michael Jackson                                         |9961245535   |184776187    |2025-11-20|2   |
|Michael Jackson        |3fMbdgg4jU18AjLCKBhRSm|HIStory - PAST, PRESENT AND FUTURE - BOOK I                             |10876505439  |165826551    |2025-11-20|3   |
|Mic

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    input_file_name, regexp_extract, to_date, col,
    regexp_replace, sum as spark_sum
)

spark = SparkSession.builder.appName("pipeline1_graph1").getOrCreate()

base_path = "/content/group_storage/APAN5400_Group-data_storage/kworb"

# 7 days of top10 albums, each file is a JSON array → multiLine=True
albums_raw = (
    spark.read
         .option("multiLine", True)
         .json(f"{base_path}/spotify_artist_top10_albums_*.json")
         .withColumn("source_file", input_file_name())
         .withColumn(
             "date_str",
             regexp_extract(
                 col("source_file"),
                 r"spotify_artist_top10_albums_(\d{8})",  # e.g. 20251201
                 1
             )
         )
         .withColumn("date", to_date("date_str", "yyyyMMdd"))
)

albums_raw.printSchema()
albums_raw.show(5, truncate=False)

root
 |-- album_title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_url: string (nullable = true)
 |-- daily: string (nullable = true)
 |-- streams: string (nullable = true)
 |-- source_file: string (nullable = false)
 |-- date_str: string (nullable = false)
 |-- date: date (nullable = true)

+-------------------------+----------------------+-----------+-------------------------------------------------------------------+----------+--------------+---------------------------------------------------------------------------------------------------------+--------+----------+
|album_title              |artist_id             |artist_name|artist_url                                                         |daily     |streams       |source_file                                                                                              |date_str|date      |
+-------------------------+----------------------+-----------+--

In [25]:
albums_clean = (
    albums_raw
    .select(
        col("artist_name"),
        col("artist_id"),
        col("album_title").alias("album_name"),
        # remove commas and cast to long
        regexp_replace("daily", ",", "").cast("long").alias("streams_daily"),
        col("date")
    )
    .dropna(subset=["artist_name", "artist_id", "album_name", "date"])
)

albums_clean.show(5, truncate=False)

+-----------+----------------------+-------------------------+-------------+----------+
|artist_name|artist_id             |album_name               |streams_daily|date      |
+-----------+----------------------+-------------------------+-------------+----------+
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|^ The Highlights (Deluxe)|21763803     |2025-12-03|
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|^ The Highlights         |13535899     |2025-12-03|
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|^ The Weeknd In Japan    |6787192      |2025-12-03|
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|Starboy (Deluxe)         |6598588      |2025-12-03|
|The Weeknd |1Xyo4u8uXC1ZmMpatF05PJ|Starboy                  |6057126      |2025-12-03|
+-----------+----------------------+-------------------------+-------------+----------+
only showing top 5 rows



In [26]:
from pyspark.sql.functions import sum as spark_sum

artist_daily_playback = (
    albums_clean
    .groupBy("artist_id", "artist_name", "date")
    .agg(
        spark_sum("streams_daily").alias("daily_playback")
    )
)

artist_daily_playback.orderBy("artist_name", "date").show(20, truncate=False)

+----------------------+-----------+----------+--------------+
|artist_id             |artist_name|date      |daily_playback|
+----------------------+-----------+----------+--------------+
|1VPmR4DJC1PlOtd0IADAO0|$uicideboy$|2025-11-30|3663193       |
|1VPmR4DJC1PlOtd0IADAO0|$uicideboy$|2025-12-01|3663193       |
|1VPmR4DJC1PlOtd0IADAO0|$uicideboy$|2025-12-02|3493246       |
|1VPmR4DJC1PlOtd0IADAO0|$uicideboy$|2025-12-03|4191579       |
|6Ff53KvcvAj5U7Z1vojB5o|*NSYNC     |2025-11-30|6961795       |
|6Ff53KvcvAj5U7Z1vojB5o|*NSYNC     |2025-12-01|6961795       |
|6Ff53KvcvAj5U7Z1vojB5o|*NSYNC     |2025-12-02|6352348       |
|6Ff53KvcvAj5U7Z1vojB5o|*NSYNC     |2025-12-03|6817970       |
|7Gy1PxqrgsiqWF6JNYPHeB|.Feast     |2025-11-30|2757174       |
|7Gy1PxqrgsiqWF6JNYPHeB|.Feast     |2025-12-01|2757174       |
|7Gy1PxqrgsiqWF6JNYPHeB|.Feast     |2025-12-02|2757174       |
|7Gy1PxqrgsiqWF6JNYPHeB|.Feast     |2025-12-03|2767789       |
|12Zk1DFhCbHY6v3xep2ZjI|070 Shake  |2025-11-30|2210947 

In [27]:
output_path_p1 = "/content/pipeline1_graph1_artist_daily_playback"

artist_daily_playback.write.mode("overwrite").option("header", True).csv(output_path_p1)

In [28]:
!ls -l /content/pipeline1_graph1_artist_daily_playback

total 520
-rw-r--r-- 1 root root 532320 Dec  4 19:43 part-00000-eb387961-8d98-4815-9d59-4ef544eff614-c000.csv
-rw-r--r-- 1 root root      0 Dec  4 19:43 _SUCCESS


In [29]:
import pandas as pd

pdf = artist_daily_playback.toPandas()
pdf.to_csv("pipeline1_graph1_artist_daily_playback.csv", index=False)

In [30]:
from google.colab import files
files.download("pipeline1_graph1_artist_daily_playback.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>